In [1]:
!pip3 install -q ktrain

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
train_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng1train.csv") #training data
test_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng1test.csv") #testing data
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_1'],test_size=0.15,random_state=42) #training and validation split
target_names=[0,1] #names of classes

In [15]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [16]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-cased' #bert predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for bert
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get bert classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for bert

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 24
	95percentile : 46
	99percentile : 52


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 23
	95percentile : 43
	99percentile : 50


In [17]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 829 steps, validate for 28 steps
Epoch 1/4
829/829 [==============================] - 651s 786ms/step - loss: 0.6442 - accuracy: 0.6301 - val_loss: 0.6122 - val_accuracy: 0.6754
Epoch 2/4
829/829 [==============================] - 639s 771ms/step - loss: 0.5908 - accuracy: 0.6896 - val_loss: 0.5757 - val_accuracy: 0.7050
Epoch 3/4
829/829 [==============================] - 639s 771ms/step - loss: 0.4965 - accuracy: 0.7700 - val_loss: 0.6145 - val_accuracy: 0.7096
Epoch 4/4
829/829 [==============================] - 640s 772ms/step - loss: 0.2213 - accuracy: 0.9212 - val_loss: 0.8666 - val_accuracy: 0.6674


In [18]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.56      0.56      0.56       330
           1       0.73      0.73      0.73       548

    accuracy                           0.67       878
   macro avg       0.65      0.65      0.65       878
weighted avg       0.67      0.67      0.67       878



array([[185, 145],
       [147, 401]])

In [19]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  bert

In [20]:
test_b_text=list(test_b['text']) #get text for testing
test_b_label=list(test_b['task_1']) #get label of testing
# val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data

In [21]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [22]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [23]:
from sklearn.metrics import classification_report

In [24]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.50      0.72      0.59       288
           1       0.89      0.77      0.82       865

    accuracy                           0.75      1153
   macro avg       0.70      0.74      0.71      1153
weighted avg       0.79      0.75      0.76      1153

